# Raw data

## Overview

In this project we use the following raw data:

* *Corine Land Cover* (CLC) from 2018 and information about the class nomenclature.

* Sentinel-2 grid

* Harmonized Landsat Sentinel-2 (HLS)

This notebook describes the raw data collection and creation process. 
Thus all the data described here can be found in the *data/raw* folder.

In [37]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely import wkt
from urllib.request import urlretrieve

import nasa_hls

from src import configs
prjconf = configs.ProjectConfigParser()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We will prepare the data for the following tiles this notebook:

In [38]:
tilenames = prjconf.get("Params", "tiles").split(" ")
tilenames

['32UNU', '32UPU', '32UQU', '33UUP']

In [39]:
tilenames =['32UNU', '32UPU', '32UQU', '33UUP', '32TPT', '32TQT', '33TUN']

### Tile grid

We download the Sentinel-2 grid in the following cell. 
The link to this nice Sentinel-2 grid file has been found on the [bencevans/sentinel-2-grid GitHub project](https://github.com/bencevans/sentinel-2-grid). 

In [4]:
url = 'https://unpkg.com/sentinel-2-grid/data/grid.json'
path__tile_grid = prjconf.get_path("Raw", "tile_grid")
path__tile_grid.parent.mkdir(exist_ok=True, parents=True)
if not path__tile_grid.exists():
    urlretrieve(url, path__tile_grid)

From this file we create single footprint file for the tile we want to process.
This is a good starting point for using Snakemake later.

In [5]:
overwrite = False
footprints_exist = [prjconf.get_path("Raw", "tile_footprint", tile).exists() for tile in tilenames]
if not all(footprints_exist):
    tile_grid = gpd.read_file(path__tile_grid)
    for tile in tilenames:
        path__tile_footprint = prjconf.get_path("Raw", "tile_footprint", tile)
        if not Path(path__tile_footprint).exists() or overwrite:
            tile = tile_grid[tile_grid["name"] == tile]
            tile = tile.to_crs(epsg=tile["epsg"].values[0])
            tile["geometry"] = tile["utmWkt"].apply(wkt.loads)
            Path(path__tile_footprint).parent.mkdir(parents=True, exist_ok=True)
            tile.to_file(path__tile_footprint, driver="GPKG")

Fast access to important parameters and file paths:

In [6]:
print(prjconf.get_path("Raw", "tile_grid"))

for tile in tilenames:
    print(prjconf.get_path("Raw", "tile_footprint", tile))

/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/tiles_grid.geojson
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_32UNU.gpkg
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_32UPU.gpkg
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_32UQU.gpkg
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_33UUP.gpkg
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_32TPT.gpkg
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_32TQT.gpkg
/home/ben/Devel/Projects/classify-hls/data/raw/footprints/tiles/footprint_33TUN.gpkg


## Create raw data

### CLC 

We downloaded *Corine Land Cover - GeoPackage* dataset manually after registration from [Copernicus Land Monitoring Service](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018?tab=download) and extracted the file into *data/raw/clc/clc2018_clc2018_v2018_20_geoPackage*.

    ### CLC - raster - OUTDATED

    We downloaded *Corine Land Cover Raster - 100m* dataset manually after registration from [Copernicus Land Monitoring Service](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018?tab=download) and extracted the file into *data/raw/clc/clc2018_clc2018_v2018_20b2_raster100m*.
    The most important file is *data/raw/clc/clc2018_clc2018_v2018_20b2_raster100m/clc2018_clc2018_V2018.20b2.tif*

    We copied the CLC legend from the *CORINE LAND COVER LEGEND* table found on the [nomenclature site of clc.gios.gov.pl](http://clc.gios.gov.pl/index.php/9-gorne-menu/clc-informacje-ogolne/58-klasyfikacja-clc-2), pasted it into LibreOffice Calc and saved it as ';'-separated csv file under *data/raw/clc/clc_legend_raw.csv*.

    An extended legend with the empty cells filled up and the level 2 and 3 class indices added is created here in the following cell and can be found under *data/raw/clc/clc_legend.csv* once the cell has been executed.

#### Legend

**TODO**: Recover all the columns as created previously:

    path__clc_legend_raw = prjconf.get_path("Raw", "rootdir") / "clc" / "clc_legend_raw.csv"
    path__clc_legend = prjconf.get_path("Raw", "rootdir") / "clc" / "clc_legend.csv"

    if not path__clc_legend.exists():
        clc_legend = pd.read_csv(path__clc_legend_raw, delimiter=";").iloc[0:44, :]
        clc_legend.columns = ["l1_name", "l2_name", "l3_name", "grid_code", "rgb"]
        clc_legend_ids = clc_legend["l3_name"].str[:5].str.split(".", expand=True)
        clc_legend["l1_id"] = clc_legend_ids[0].astype("uint8")
        clc_legend["l2_id"] = (clc_legend_ids[0] + clc_legend_ids[1]).astype("uint8")
        clc_legend["l3_id"] = (clc_legend_ids[0] + clc_legend_ids[1] + clc_legend_ids[2]).astype("int")
        clc_legend["l1_name"] = clc_legend["l1_name"].str[3::]
        clc_legend["l2_name"] = clc_legend["l2_name"].str[4::]
        clc_legend["l3_name"] = clc_legend["l3_name"].str[6::]
        clc_legend = clc_legend.fillna(method="ffill")
        clc_legend.to_csv(path__clc_legend, index=False)

In [7]:
clc_legend = prjconf.get_clc_legend()
clc_legend

,grid_code,cid_l3,name_l3,rgb,cid_l2,cid_l1
0,1,111,Continuous urban fabric,230-000-077,11,1
1,2,112,Discontinuous urban fabric,255-000-000,11,1
2,3,121,Industrial or commercial units,204-077-242,12,1
3,4,122,Road and rail networks and associated land,204-000-000,12,1
4,5,123,Port areas,230-204-204,12,1
5,6,124,Airports,230-204-230,12,1
6,7,131,Mineral extraction sites,166-000-204,13,1
7,8,132,Dump sites,166-077-000,13,1
8,9,133,Construction sites,255-077-255,13,1
9,10,141,Green urban areas,255-166-255,14,1


#### Vector data

The most important file is *data/raw/clc/clc2018_clc2018_v2018_20_geoPackage/CLC2018_CLC2018_V2018_20.gpkg*.

Let's first load the whole data and convert the ID column to a integer ID column (from *EU-< ID >* to *< ID >*).

Then, we loop through the previously created tiles and select the polygons which are within the respective tile.

We save all polygons and the ones with a area less then or equal to 50 ha and 100 ha as separate files.

In [10]:
path__clc_complete = prjconf.get_path("Raw", "clc_complete")
clc = gpd.read_file(path__clc_complete)
assert clc.ID.is_unique
clc["pid"] = clc.ID.str.split("-", expand=True)[1].astype(int)
assert clc.pid.is_unique
display(clc.head())

,Code_18,Remark,Area_Ha,ID,geometry,pid
0,111,None,130.863654,EU-1,"(POLYGON Z ((1917182.16 943608.8599999994 0, 1...",1
1,111,None,53.744524,EU-2,"(POLYGON Z ((1953122.84 950507.4399999995 0, 1...",2
2,111,None,30.719104,EU-3,"(POLYGON Z ((1956709.15 951094.5500000007 0, 1...",3
3,111,None,50.201782,EU-4,"(POLYGON Z ((1805587.5 950821.5399999991 0, 18...",4
4,111,None,481.848803,EU-5,"(POLYGON Z ((1792547.84 952643.3800000008 0, 1...",5


In [35]:
for tilename in tilenames:
    print("*" * 80)
    print(tilename)
    path__clc = prjconf.get_path("Raw", "clc", tilename)
    path__clc_lte50ha = prjconf.get_path("Raw", "clc_lte50ha", tilename)
    path__clc_lte100ha = prjconf.get_path("Raw", "clc_lte100ha", tilename)
    
    # MORE PERFORMANT SOLUTION
    # https://gis.stackexchange.com/questions/270043/select-by-location-using-ogr2ogr-sqlite

    # EASY SOLUTION - WHAT WE DO HERE
    # https://gis.stackexchange.com/questions/279670/geopandas-equivalent-to-select-by-location
    path__tile_footprint = prjconf.get_path("Raw", "tile_footprint", tilename)
    tile = gpd.read_file(path__tile_footprint)
    tile_original_crs = tile.crs.copy()
    tile = tile.to_crs(clc.crs)
        
    idx = clc.within(tile.geometry[0])
    print(f"Number of polygons within the tile          : {idx.sum()}")
    clc_tile = clc[idx]

    clc_tile["Code_18"]  = clc_tile["Code_18"].astype(int)
    clc_tile = pd.merge(clc_tile, 
                        clc_legend[["grid_code", "cid_l3", "cid_l2", "cid_l1"]], 
                        how="left", left_on="Code_18", right_on="cid_l3")

    clc_tile = clc_tile[["pid", "cid_l3", "cid_l2", "cid_l1", "Area_Ha", "geometry"]]

    clc_tile = clc_tile.to_crs(tile_original_crs)
    clc_tile.to_file(path__clc, "GPKG")

    idx = clc_tile["Area_Ha"] <= 100
    print(f"Number of polygons <= 100 ha within the tile: {idx.sum()}")
    clc_tile_lte100ha = clc_tile[idx]
    clc_tile_lte100ha.to_file(path__clc_lte100ha, "GPKG")

    idx = clc_tile["Area_Ha"] <= 50
    print(f"Number of polygons <= 50 ha within the tile : {idx.sum()}")
    clc_tile_lte50ha = clc_tile[idx]
    clc_tile_lte50ha.to_file(path__clc_lte50ha, "GPKG")


********************************************************************************
32UNU
Number of polygons within the tile          : 6963


/home/ben/anaconda3/envs/classify-hls/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Number of polygons <= 100 ha within the tile: 5036
Number of polygons <= 50 ha within the tile : 3106
********************************************************************************
32UPU
Number of polygons within the tile          : 5896
Number of polygons <= 100 ha within the tile: 4370
Number of polygons <= 50 ha within the tile : 2707
********************************************************************************
32UQU
Number of polygons within the tile          : 5955
Number of polygons <= 100 ha within the tile: 4412
Number of polygons <= 50 ha within the tile : 2719
********************************************************************************
33UUP
Number of polygons within the tile          : 6345
Number of polygons <= 100 ha within the tile: 4512
Number of polygons <= 50 ha within the tile : 2635
********************************************************************************
32TPT
Number of polygons within the tile          : 4914
Number of polygons <= 100 ha within the 

Fast access to important file paths:

### HLS

We download data from the [Harmonized Landsat Sentinel-2 (HLS) Product](https://hls.gsfc.nasa.gov/) with the [nasa_hls Python package](https://benmack.github.io/nasa_hls/build/html/index.html) in the following cell.

In [42]:
for tile in tilenames:
    df_datasets = nasa_hls.get_available_datasets(products=["L30"],
                                                  years=[2018],
                                                  tiles=[tile],
                                                  return_list=False)
    print(f"Number of scenes queried for tile {tile}: {df_datasets.shape[0]}")
    dir__hls_tile = prjconf.get_path("Raw", "hls_tile", tile=tile)
    nasa_hls.download_batch(dir__hls_tile, df_datasets)
    
    path__hls_tile_lut = prjconf.get_path("Raw", "hls_tile_lut", tile=tile)
    if not path__hls_tile_lut.exists():
        hdf_files = list(dir__hls_tile.rglob("*.hdf"))
        df = nasa_hls.dataframe_from_hdf_paths(hdf_files)
        df["tile"] = tile
        df.to_csv(path__hls_tile_lut, index=False)
    else:
        pass 
        # TODO: it would be good to check if there are new files and rewrite the csv ONLY if this is the case 

  0%|          | 0/1 [00:00<?, ?it/s]

Number of scenes queried for tile 32UNU: 67


  0%|          | 0/1 [00:00<?, ?it/s]

Number of scenes queried for tile 32UPU: 67


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Number of scenes queried for tile 32TQT: 68


  0%|          | 0/49 [00:00<?, ?it/s]

Number of scenes queried for tile 33TUN: 49


100%|██████████| 49/49 [01:08<00:00,  1.40s/it]


Fast access to important directories:

In [43]:
for tile in tilenames:
    print(prjconf.get_path("Raw", "hls_tile", tile))
    print(prjconf.get_path("Raw", "hls_tile_lut", tile=tile))

/home/ben/Devel/Projects/classify-hls/data/raw/hls/32UNU
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_32UNU_lut.csv
/home/ben/Devel/Projects/classify-hls/data/raw/hls/32UPU
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_32UPU_lut.csv
/home/ben/Devel/Projects/classify-hls/data/raw/hls/32UQU
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_32UQU_lut.csv
/home/ben/Devel/Projects/classify-hls/data/raw/hls/33UUP
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_33UUP_lut.csv
/home/ben/Devel/Projects/classify-hls/data/raw/hls/32TPT
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_32TPT_lut.csv
/home/ben/Devel/Projects/classify-hls/data/raw/hls/32TQT
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_32TQT_lut.csv
/home/ben/Devel/Projects/classify-hls/data/raw/hls/33TUN
/home/ben/Devel/Projects/classify-hls/data/raw/hls/hls_33TUN_lut.csv
